In [1]:
import pandas as pd
# from google.colab import drive
import json
from geopy.distance import geodesic
import plotly.express as px
import plotly.graph_objects as go
import os

In [2]:
# If using google drive comment/uncomment
# drive.mount('/content/drive')
# BASE_PATH = '/content/drive/MyDrive/almrrc2021/almrrc2021-data-training/model_build_inputs/'

# If you have the data locally
BASE_PATH = '../almrrc2021/almrrc2021-data-training/model_build_inputs/'

# If using data sample
# BASE_PATH = 'data/'

In [3]:
# Loading all the package data

package_data = json.load(open(BASE_PATH + 'package_data.json'))

flattened_package_data = []
for route_id, stops in package_data.items():
    for stop_id, packages in stops.items():
        for package_id, details in packages.items():
            record = {
                "route_id": route_id,
                "stop_id": stop_id,
                "package_id": package_id,
                "scan_status": details["scan_status"] if "scan_status" in details else None, # note model_apply_inputs does not have scan_status, model_build_inputs does
                "start_time_utc": details["time_window"]["start_time_utc"],
                "end_time_utc": details["time_window"]["end_time_utc"],
                "planned_service_time_seconds": details["planned_service_time_seconds"],
                "depth_cm": details["dimensions"]["depth_cm"],
                "height_cm": details["dimensions"]["height_cm"],
                "width_cm": details["dimensions"]["width_cm"],
            }
            flattened_package_data.append(record)

package_data_df = pd.DataFrame(flattened_package_data)

package_data_df.head()

,route_id,stop_id,package_id,scan_status,start_time_utc,end_time_utc,planned_service_time_seconds,depth_cm,height_cm,width_cm
0,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,AD,PackageID_9d7fdd03-f2cf-4c6f-9128-028258fc09ea,DELIVERED,NaN,NaN,59.3,25.4,7.6,17.8
1,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,AD,PackageID_5541e679-b7bd-4992-b288-e862f6c84ae7,DELIVERED,2018-07-27 16:00:00,2018-07-28 00:00:00,59.3,25.4,12.7,17.8
2,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,AD,PackageID_84d0295b-1adb-4a33-a65e-f7d6247c7a07,DELIVERED,NaN,NaN,59.3,39.4,7.6,31.8
3,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,AF,PackageID_15c6a204-ec5f-4ced-9c3d-472316cc7759,DELIVERED,2018-07-27 16:00:00,2018-07-28 00:00:00,27.0,30.0,3.0,27.4
4,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,AG,PackageID_3b28f781-242e-416e-9575-84c7188b8208,DELIVERED,NaN,NaN,45.0,25.4,12.7,17.8


In [4]:
# Loading all the route data

route_data = json.load(open(BASE_PATH + 'route_data.json'))

flattened_route_data = []

for route_id, info in route_data.items():
    for stop_id, stop_details in info['stops'].items():
        flattened_route_data.append({
            "route_id": route_id,
            "station_code": info['station_code'],
            "date": info['date_YYYY_MM_DD'],
            "departure_time_utc": info['departure_time_utc'],
            "executor_capacity_cm3": info['executor_capacity_cm3'],
            "route_score": info['route_score'],
            "stop_id": stop_id,
            "lat": stop_details['lat'],
            "lng": stop_details['lng'],
            "type": stop_details['type'],
            "zone_id": stop_details['zone_id']
        })

route_data_df = pd.DataFrame(flattened_route_data)

route_data_df.head()

,route_id,station_code,date,departure_time_utc,executor_capacity_cm3,route_score,stop_id,lat,lng,type,zone_id
0,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,DLA3,2018-07-27,16:02:10,3313071.0,High,AD,34.099611,-118.283062,Dropoff,P-12.3C
1,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,DLA3,2018-07-27,16:02:10,3313071.0,High,AF,34.101587,-118.291125,Dropoff,A-1.2D
2,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,DLA3,2018-07-27,16:02:10,3313071.0,High,AG,34.089727,-118.285530,Dropoff,A-2.1A
3,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,DLA3,2018-07-27,16:02:10,3313071.0,High,BA,34.096132,-118.292869,Dropoff,A-1.2C
4,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,DLA3,2018-07-27,16:02:10,3313071.0,High,BE,34.098482,-118.286243,Dropoff,P-13.3B


In [5]:
# Loading all the travel times

vehicle_travel_times = json.load(open(BASE_PATH + 'travel_times.json'))

**To create a solution to the VRPD for we need the following information for a route:**



We already have:

Package Data
*   Package ID
*   Dimensions (length, width, height)
*   Service time seconds

Stop Data
*   Stop ID
*   Latitude and Longitude
*   Zone ID
*   List of Package ID(s) to be delivered
*   Depot bool

Travel Times:
*   Vehicle transit time to every other stop


We will need to calculate
*   Drone bool for each stop ((if a drone can cover this stop ie. one small package)
* Drone transit time to every other stop



We can begin by collecting all the data we want for a desired route





In [6]:
def get_data_for_route(route_id):
    package_data = package_data_df[package_data_df['route_id'] == route_id]
    route_data = route_data_df[route_data_df['route_id'] == route_id]
    travel_times_route = vehicle_travel_times[route_id]

    return package_data.copy(), route_data.copy(), travel_times_route.copy()

In [7]:
# Given route data, it will return a dictionary
def calculate_drone_travel_time(route_data):
  drone_travel_time = {}

  for _, base_row in route_data.iterrows():
    drone_travel_time[base_row['stop_id']] = {}

    # calculate travel time to every other stop in the route
    for _, other_row in route_data.iterrows():
      if base_row['stop_id'] != other_row['stop_id']:
        base_location = (base_row['lat'], base_row['lng'])
        other_location = (other_row['lat'], other_row['lng'])

        distance = geodesic(base_location, other_location).meters

        # Assuming the drone will travel 7 m/s TODO: CONSTANT
        drone_travel_time[base_row['stop_id']][other_row['stop_id']] = distance / 7.0
        # drone_travel_time[base_row['stop_id']][other_row['stop_id']] += 10 # 5 second rise, 5 second drop

      else:
        drone_travel_time[base_row['stop_id']][other_row['stop_id']] = 0

  return drone_travel_time

In [8]:
package_data_route_df, route_data_route_df, vehicle_travel_times_route = get_data_for_route("RouteID_f9639176-8909-4c65-80a9-5562b0241ad4")

In [9]:
drone_travel_times_route = calculate_drone_travel_time(route_data_route_df)

In [10]:
# Determining if each package can be delivered by a drone
package_data_route_df['drone_possible'] = (package_data_route_df['depth_cm'] < 30) & (package_data_route_df['height_cm'] < 25) & (package_data_route_df['width_cm'] < 30) # TODO: be indifferent on depth and width

In [11]:
package_info_agg = package_data_route_df.groupby('stop_id').apply(
    lambda x: pd.Series({
        'packages': x[['package_id', 'depth_cm', 'height_cm', 'width_cm']].to_dict('records'),
        'num_packages': len(x),
        'drone_possible': (len(x) == 1) and (x['drone_possible'].iloc[0] == True),
        'service_time_seconds': x['planned_service_time_seconds'].mean() # Not sure if this should be sum or average. https://github.com/MIT-CAVE/rc-cli/blob/main/templates/data_structures.md
    })
).reset_index()

In [12]:
route_data_route_df = pd.merge(route_data_route_df, package_info_agg, how='left', on='stop_id')

In [13]:
route_data_route_df.head()

,route_id,station_code,date,departure_time_utc,executor_capacity_cm3,route_score,stop_id,lat,lng,type,zone_id,packages,num_packages,drone_possible,service_time_seconds
0,RouteID_f9639176-8909-4c65-80a9-5562b0241ad4,DBO3,2018-08-13,13:02:17,3313071.0,Medium,AC,42.131001,-71.538114,Dropoff,E-1.2J,[{'package_id': 'PackageID_469f7aa0-e0db-474e-...,1.0,True,65.0
1,RouteID_f9639176-8909-4c65-80a9-5562b0241ad4,DBO3,2018-08-13,13:02:17,3313071.0,Medium,AD,42.110252,-71.555689,Dropoff,E-2.3H,[{'package_id': 'PackageID_caa3dc05-b659-4228-...,1.0,False,75.0
2,RouteID_f9639176-8909-4c65-80a9-5562b0241ad4,DBO3,2018-08-13,13:02:17,3313071.0,Medium,AG,42.138349,-71.542410,Dropoff,E-1.2H,[{'package_id': 'PackageID_32227deb-67a8-4c65-...,2.0,False,62.5
3,RouteID_f9639176-8909-4c65-80a9-5562b0241ad4,DBO3,2018-08-13,13:02:17,3313071.0,Medium,AL,42.084637,-71.565057,Dropoff,E-2.1H,[{'package_id': 'PackageID_fe18bcf4-808c-4b64-...,1.0,False,140.0
4,RouteID_f9639176-8909-4c65-80a9-5562b0241ad4,DBO3,2018-08-13,13:02:17,3313071.0,Medium,AP,42.128049,-71.578627,Dropoff,E-2.2J,[{'package_id': 'PackageID_5d77652d-1f5f-40fd-...,1.0,True,100.0


## Naive Solver
This solver will just go to the next closest stop within the Zone, and then the next closest stop in a different zone.

In [14]:
# Event Types

# Both
LOAD_START = 'LOAD_START'
LOAD_END = 'LOAD_END'
ARRIVE = 'ARRIVE'
DEPART = 'DEPART'

# Vehicle
SERVICE_START = 'SERVICE_START'
SERVICE_END = 'SERVICE_END'


# Drone
LAUNCH = 'LAUNCH'
DRONE_DELIVERY_START = 'DRONE_DELIVERY_START'
DRONE_DELIVERY_END = 'DRONE_DELIVERY_END'
LAND = 'LAND'

In [15]:
class Event:
  def __init__(self, event_type, stop_id, time, data): # data holds extra information, like coordinates
    self.event_type = event_type
    self.stop_id = stop_id
    self.time = time
    self.data = data

  def __str__(self):
    return f"{self.event_type} {self.stop_id} {round(self.time, 1)} {self.data}"

  def to_dict(self):
    # Convert the event to a dictionary
    return {
        'event_type': self.event_type,
        'stop_id': self.stop_id,
        'time': self.time,
        'coordinates': self.data,
    }

In [46]:
class VRPDSolver:
  def __init__(self, route_id, package_data, route_data, vehicle_travel_times, drone_travel_times):
    self.route_id = route_id
    self.package_data = package_data
    self.route_data = route_data
    self.vehicle_travel_times = vehicle_travel_times
    self.drone_travel_times = drone_travel_times
    self.station = self.route_data[self.route_data['type'] == 'Station'].iloc[0]

    self.vehicle_path = []
    self.vehicle_events = []
    self.drone_path = []
    self.drone_events = []

    self.remaining_stops = set(self.route_data['stop_id'].unique())
    self.completed_stops = set()


    self.cur_stop = None
    self.cur_zone = None
    self.cur_coords = None
    self.cur_time = 0
    self.final_time = 0

  # Get closest stop
  def get_closest_stop(self):

    possible_zones = None
    if self.cur_zone:
      # Get all possible zones
      possible_zones = self.route_data[self.route_data['zone_id'] == self.cur_zone]
      # Remove any zones that have been visited
      possible_zones = possible_zones[~possible_zones['stop_id'].isin(self.completed_stops)]

    # If there are more stops left in the zone, we find the closest one using it's stop_id and vehicle_travel_times
    best_next_stop = None
    cur_min_time = float('inf')

    # We find the next closest stop in the zone
    if possible_zones is not None and not possible_zones.empty:
      for _, row in possible_zones.iterrows():
        if self.vehicle_travel_times[self.cur_stop][row['stop_id']] < cur_min_time and row['stop_id'] != self.cur_stop and row['stop_id'] !=  self.station['stop_id']:
          cur_min_time = self.vehicle_travel_times[self.cur_stop][row['stop_id']]
          best_next_stop = row['stop_id']
    
    if best_next_stop is None:
      # We find the next closest stop that has not been visited
      for stop in self.remaining_stops:
          if self.vehicle_travel_times[self.cur_stop][stop] < cur_min_time and stop != self.cur_stop:
            cur_min_time = self.vehicle_travel_times[self.cur_stop][stop]
            best_next_stop = stop

    if best_next_stop is None:
      print("setting")
      best_next_stop = self.station['stop_id']

    return best_next_stop

  def naive_vrp(self):
    
    # The first step is departing the station
    self.cur_stop = self.station['stop_id']
    self.cur_coords = {'lat' : self.station['lat'], 'lng' : self.station['lng']}

    # We leave the depot station
    first_event = Event(DEPART, self.cur_stop, self.cur_time, self.cur_coords)
    self.vehicle_events.append(first_event)
    self.vehicle_path.append(self.cur_stop)

    while len(self.remaining_stops) > 0:
     
      # We find the closest stop
      next_stop = self.get_closest_stop()
      # Travel there
      self.cur_time += self.vehicle_travel_times[self.cur_stop][next_stop]
      # Arrive
      self.cur_stop = next_stop
      stop = self.route_data[self.route_data['stop_id'] == self.cur_stop].iloc[0]
      self.cur_zone = stop['zone_id']
      self.cur_coords = {'lat' : stop['lat'], 'lng' : stop['lng']}
      arrive_event = Event(ARRIVE, self.cur_stop, self.cur_time, self.cur_coords)
      self.vehicle_events.append(arrive_event)
      self.vehicle_path.append(self.cur_stop)

      # # Edge case: If we arrive at the station we can break before delivering packages
      if self.cur_stop == self.station['stop_id']:
        print(self.cur_stop)
        print(len(self.remaining_stops))
        break

      # Driver delivers package
      service_start_event = Event(SERVICE_START, self.cur_stop, self.cur_time, self.cur_coords)
      self.vehicle_events.append(service_start_event)
      self.cur_time += stop['service_time_seconds']
      service_end_event = Event(SERVICE_END, self.cur_stop, self.cur_time, self.cur_coords)
      self.vehicle_events.append(service_end_event)
      self.completed_stops.add(self.cur_stop)
      self.remaining_stops.remove(self.cur_stop)
      # Driver departs
      depart_event = Event(DEPART, self.cur_stop, self.cur_time, self.cur_coords)
      self.vehicle_events.append(depart_event)

    self.final_time = self.cur_time


  # Takes in three stops and determines if a drone should be used for the middle stop
  def augment_good(self, next_stops):
      # We check if a drone can deliver to the middle stop
      mid_stop = self.route_data[self.route_data['stop_id'] == next_stops[1]].iloc[0]
      if mid_stop["drone_possible"]:
      # We see if the timing is bett
        vehicle_time = 0
        drone_time = 0
        vehicle_drone_time = 0

        vehicle_time += self.vehicle_travel_times[next_stops[0]][next_stops[1]]
        vehicle_time += self.vehicle_travel_times[next_stops[1]][next_stops[2]]
        vehicle_time += mid_stop['service_time_seconds']

        drone_time += 15 # For loading the drone
        drone_time += 5 # For launching the drone
        drone_time += self.drone_travel_times[next_stops[0]][next_stops[1]]
        drone_time += 10 # Drone landing, delivering, launching
        drone_time += self.drone_travel_times[next_stops[1]][next_stops[2]]
        drone_time += 5 # For landing the drone

        vehicle_drone_time += 15 # For loading the drone
        vehicle_drone_time += self.vehicle_travel_times[next_stops[0]][next_stops[2]]

        max_time = max(vehicle_drone_time, drone_time)
        # print(f"Vehicle time: {vehicle_time}")
        # print(f"Drone time: {drone_time}")
        # print(f"Vehicle drone time: {vehicle_drone_time}")
        # print("------------")
        return max_time < vehicle_time
      else:
        return False

  def naive_vrpd(self):
    # The first step is departing the station
    self.cur_stop = self.station['stop_id']
    self.cur_coords = {'lat' : self.station['lat'], 'lng' : self.station['lng']}

    # We leave the depot station
    first_event = Event(DEPART, self.cur_stop, self.cur_time, self.cur_coords)
    self.vehicle_events.append(first_event)
    self.vehicle_path.append(self.cur_stop)
    self.drone_events.append(first_event)
    self.drone_path.append(self.cur_stop)


    while len(self.remaining_stops) > 0:
      # We find the next three stops for the vehicle
      # If possible we agument with possible and shorter time we augment with a drone

      # Doesn't account for differnet iterations, but simple is fine for this

      base_stop = self.cur_stop # What we will default to
      next_stops = []
      drone_attempt = len(self.remaining_stops) > 4

      if drone_attempt:
        for i in range(3):
          next_stop = self.get_closest_stop()
          self.cur_stop = next_stop
          next_stops.append(next_stop)
          if i != 2:
            self.completed_stops.add(self.cur_stop)
            self.remaining_stops.remove(self.cur_stop)

      # We augment by using drone
      if drone_attempt and self.augment_good(next_stops):
        next_stop = next_stops[0]
        self.cur_time += self.vehicle_travel_times[self.cur_stop][next_stop]
        # Arrive at first stop
        self.cur_stop = next_stop
        stop = self.route_data[self.route_data['stop_id'] == self.cur_stop].iloc[0]
        self.cur_zone = stop['zone_id']
        self.cur_coords = {'lat' : stop['lat'], 'lng' : stop['lng']}
        arrive_event = Event(ARRIVE, self.cur_stop, self.cur_time, self.cur_coords)
        self.vehicle_events.append(arrive_event)
        self.vehicle_path.append(self.cur_stop)
        self.drone_events.append(arrive_event)
        self.drone_path.append(self.cur_stop)

        # Driver delivers package
        service_start_event = Event(SERVICE_START, self.cur_stop, self.cur_time, self.cur_coords)
        self.vehicle_events.append(service_start_event)
        self.cur_time += stop['service_time_seconds']
        service_end_event = Event(SERVICE_END, self.cur_stop, self.cur_time, self.cur_coords)
        self.vehicle_events.append(service_end_event)

        # Drive loads the drone
        load_drone_event = Event(LOAD_START, self.cur_stop, self.cur_time, self.cur_coords)
        self.vehicle_events.append(load_drone_event)
        self.drone_events.append(load_drone_event)
        self.cur_time += 15
        load_drone_event = Event(LOAD_END, self.cur_stop, self.cur_time, self.cur_coords)
        self.vehicle_events.append(load_drone_event)
        self.drone_events.append(load_drone_event)

        #TODO: Add a class variable to keep track of drone logistics
        drone_time = self.cur_time

        # Drone Leaves to deliver
        drone_launch_event = Event(LAUNCH, self.cur_stop, drone_time, self.cur_coords)
        self.drone_events.append(drone_launch_event)
        drone_time += 5
        drone_time += self.drone_travel_times[next_stops[0]][next_stops[1]] # Travels to middle stop
        stop = self.route_data[self.route_data['stop_id'] == next_stops[1]].iloc[0]
        drone_coords = {'lat' : stop['lat'], 'lng' : stop['lng']}
        drone_arrive_event = Event(DRONE_DELIVERY_START, next_stops[1], drone_time, drone_coords)
        self.drone_events.append(drone_arrive_event)
        drone_time += 10 # Drone delivering
        self.drone_path.append(next_stops[1])
        drone_depart_event = Event(DRONE_DELIVERY_END, next_stops[1], drone_time, drone_coords)
        self.drone_events.append(drone_depart_event)
        drone_time += self.drone_travel_times[next_stops[1]][next_stops[2]] # Travels to last stop

        # Before the drone lands, we have to ensure that the vehicle is there

        # Vehicle departs
        vehicle_depart_event = Event(DEPART, self.cur_stop, self.cur_time, self.cur_coords)
        self.vehicle_events.append(vehicle_depart_event)

        self.cur_time += self.vehicle_travel_times[next_stops[0]][next_stops[2]] # Traveling to next stop
        # Vehicle arrives
        self.cur_stop = next_stops[2]
        stop = self.route_data[self.route_data['stop_id'] == self.cur_stop].iloc[0]
        self.cur_zone = stop['zone_id']
        self.cur_coords = {'lat' : stop['lat'], 'lng' : stop['lng']}
        vehicle_arrive_event = Event(ARRIVE, self.cur_stop, self.cur_time, self.cur_coords)
        self.vehicle_events.append(vehicle_arrive_event)
        self.vehicle_path.append(self.cur_stop)

        # If the vehicle can land it tries to do so
        landed = False
        if drone_time < self.cur_time:
          landed = True
          drone_land_event = Event(LAND, self.cur_stop, self.cur_time, self.cur_coords)
          self.drone_events.append(drone_land_event)
          self.drone_path.append(self.cur_stop)

        # Driver Delivers Package
        service_start_event = Event(SERVICE_START, self.cur_stop, self.cur_time, self.cur_coords)
        self.vehicle_events.append(service_start_event)
        self.cur_time += stop['service_time_seconds']
        service_end_event = Event(SERVICE_END, self.cur_stop, self.cur_time, self.cur_coords)

        self.vehicle_events.append(service_end_event)
        self.completed_stops.add(self.cur_stop)
        self.remaining_stops.remove(self.cur_stop)

        # Drone tries to land again, makes vehicle wait if needed
        if not landed:
          if drone_time < self.cur_time:
            drone_land_event = Event(LAND, self.cur_stop, drone_time, self.cur_coords) # Note we now land on drone_time
            self.drone_events.append(drone_land_event)
            self.drone_path.append(self.cur_stop)
          else:
            self.cur_time = max(self.cur_time, drone_time)
            drone_land_event = Event(LAND, self.cur_stop, self.cur_time, self.cur_coords)
            self.drone_events.append(drone_land_event)
            self.drone_path.append(self.cur_stop)

        # Driver departs
        depart_event = Event(DEPART, self.cur_stop, self.cur_time, self.cur_coords)
        self.vehicle_events.append(depart_event)
        self.drone_events.append(depart_event)


      else:

        # We reverse the affects of finding the 3 next stops
        self.cur_stop = base_stop
        if next_stops:
          self.completed_stops.remove(next_stops[0])
          self.completed_stops.remove(next_stops[1])
          self.remaining_stops.add(next_stops[0])
          self.remaining_stops.add(next_stops[1])

        # We find the closest stop
        next_stop = self.get_closest_stop()
        # Travel there
        self.cur_time += self.vehicle_travel_times[self.cur_stop][next_stop]
        # Arrive
        self.cur_stop = next_stop
        stop = self.route_data[self.route_data['stop_id'] == self.cur_stop].iloc[0]
        self.cur_zone = stop['zone_id']
        self.cur_coords = {'lat' : stop['lat'], 'lng' : stop['lng']}
        arrive_event = Event(ARRIVE, self.cur_stop, self.cur_time, self.cur_coords)
        self.vehicle_events.append(arrive_event)
        self.vehicle_path.append(self.cur_stop)
        self.drone_path.append(self.cur_stop)
        self.drone_events.append(arrive_event)

        # # Edge case: If we arrive at the station we can break before delivering packages
        if self.cur_stop == self.station['stop_id']:
          
          break

        # Driver delivers package
        service_start_event = Event(SERVICE_START, self.cur_stop, self.cur_time, self.cur_coords)
        self.vehicle_events.append(service_start_event)
        self.cur_time += stop['service_time_seconds']
        service_end_event = Event(SERVICE_END, self.cur_stop, self.cur_time, self.cur_coords)
        self.vehicle_events.append(service_end_event)
        self.completed_stops.add(self.cur_stop)
        self.remaining_stops.remove(self.cur_stop)
        # Driver departs
        depart_event = Event(DEPART, self.cur_stop, self.cur_time, self.cur_coords)
        self.vehicle_events.append(depart_event)
        self.drone_events.append(depart_event)

      self.final_time = self.cur_time



  def print_vehicle_path(self):
          print(self.vehicle_path)  # Ensure vehicle_path is correctly converted to string

  def print_vehicle_events(self):
          for event in self.vehicle_events:
              print(event)  # Convert Event object to string

  def print_drone_path(self):
          print(self.drone_path)  # Ensure drone_path is correctly converted to string

  def print_drone_events(self):
          for event in self.drone_events:
              print(event)

  def save_vehicle_path(self, path):
    os.makedirs(path, exist_ok=True)
    path_json = []
    for i, stop_id in enumerate(self.vehicle_path):
      stop = self.route_data[self.route_data['stop_id'] == stop_id].iloc[0]
      vehicle_coords = {'lat' : stop['lat'], 'lng' : stop['lng']}
      path_json.append({"id": i+1, "stop": stop_id, "coordinates": vehicle_coords})

    with open(f"{path}/{self.route_id}_vehicle_path.json", 'w') as file:
        json.dump(path_json, file, indent=4)

  def save_drone_path(self, path):
    os.makedirs(path, exist_ok=True)
    path_json = []
    for i, stop_id in enumerate(self.drone_path):
      stop = self.route_data[self.route_data['stop_id'] == stop_id].iloc[0]
      drone_coords = {'lat' : stop['lat'], 'lng' : stop['lng']}
      path_json.append({"id": i+1, "stop": stop_id, "coordinates": drone_coords})

    with open(f"{path}/{self.route_id}_drone_path.json", 'w') as file:
        json.dump(path_json, file, indent=4)

  def save_vehicle_events(self, path):
    os.makedirs(path, exist_ok=True)
    # Convert the list of events to a list of dictionaries and assign an ID to each
    events_with_ids = [{'id': i+1, **event.to_dict()} for i, event in enumerate(self.vehicle_events)]
    # Save the list of events as a JSON file
    with open(f"{path}/{self.route_id}_vehicle_events.json", 'w') as file:
        json.dump(events_with_ids, file, indent=4)

  def save_drone_events(self, path):
    os.makedirs(path, exist_ok=True)
    # Convert the list of events to a list of dictionaries and assign an ID to each
    events_with_ids = [{'id': i+1, **event.to_dict()} for i, event in enumerate(self.drone_events)]
    # Save the list of events as a JSON file
    with open(f"{path}/{self.route_id}_drone_events.json", 'w') as file:
        json.dump(events_with_ids, file, indent=4)



In [ ]:
route_id = "RouteID_0bce0fe7-3ec0-41d8-8f90-3c766c493da1"
package_data_route_df, route_data_route_df, vehicle_travel_times_route = get_data_for_route(route_id)
route_data_route_df['zone_id'].ffill(inplace=True)

drone_travel_times_route = calculate_drone_travel_time(route_data_route_df)


package_data_route_df['drone_possible'] = (package_data_route_df['depth_cm'] < 30) & (package_data_route_df['height_cm'] < 25) & (package_data_route_df['width_cm'] < 30) # TODO: be indifferent on depth and width



package_info_agg = package_data_route_df.groupby('stop_id').apply(
lambda x: pd.Series({
    'packages': x[['package_id', 'depth_cm', 'height_cm', 'width_cm']].to_dict('records'),
    'num_packages': len(x),
    'drone_possible': (len(x) == 1) and (x['drone_possible'].iloc[0] == True),
    'service_time_seconds': x['planned_service_time_seconds'].mean() # Not sure if this should be sum or average. https://github.com/MIT-CAVE/rc-cli/blob/main/templates/data_structures.md
})
).reset_index()

route_data_route_df = pd.merge(route_data_route_df, package_info_agg, how='left', on='stop_id')



    

In [47]:
solver = VRPDSolver(route_id, package_data_route_df, route_data_route_df, vehicle_travel_times_route, drone_travel_times_route)
solver.naive_vrpd()
# save everything
solver.save_vehicle_path('test')
solver.save_drone_events('test')

In [48]:
print(len(route_data_route_df))
print(len(solver.vehicle_path))

146
129


In [49]:
solver.print_vehicle_events()

DEPART DI 0 {'lat': 33.688122, 'lng': -117.847178}
ARRIVE TQ 50.5 {'lat': 33.570311, 'lng': -117.73271}
SERVICE_START TQ 50.5 {'lat': 33.570311, 'lng': -117.73271}
SERVICE_END TQ 131.5 {'lat': 33.570311, 'lng': -117.73271}
LOAD_START TQ 131.5 {'lat': 33.570311, 'lng': -117.73271}
LOAD_END TQ 146.5 {'lat': 33.570311, 'lng': -117.73271}
DEPART TQ 146.5 {'lat': 33.570311, 'lng': -117.73271}
ARRIVE JY 187.0 {'lat': 33.569678, 'lng': -117.731179}
SERVICE_START JY 187.0 {'lat': 33.569678, 'lng': -117.731179}
SERVICE_END JY 230.0 {'lat': 33.569678, 'lng': -117.731179}
DEPART JY 230.0 {'lat': 33.569678, 'lng': -117.731179}
ARRIVE EY 249.5 {'lat': 33.56864, 'lng': -117.731061}
SERVICE_START EY 249.5 {'lat': 33.56864, 'lng': -117.731061}
SERVICE_END EY 291.5 {'lat': 33.56864, 'lng': -117.731061}
DEPART EY 291.5 {'lat': 33.56864, 'lng': -117.731061}
ARRIVE FE 314.5 {'lat': 33.568695, 'lng': -117.731665}
SERVICE_START FE 314.5 {'lat': 33.568695, 'lng': -117.731665}
SERVICE_END FE 361.5 {'lat': 33.

In [ ]:
test.print_drone_events()

In [ ]:
route_data_ordered = pd.concat([route_data_route_df[route_data_route_df['stop_id'] == stop_id] for stop_id in test.vehicle_path]).reset_index(drop=True)
route_data_ordered.head()

,route_id,station_code,date,departure_time_utc,executor_capacity_cm3,route_score,stop_id,lat,lng,type,zone_id,packages,num_packages,drone_possible,service_time_seconds
0,RouteID_f9639176-8909-4c65-80a9-5562b0241ad4,DBO3,2018-08-13,13:02:17,3313071.0,Medium,BU,42.139891,-71.494346,Station,NaN,NaN,NaN,NaN,NaN
1,RouteID_f9639176-8909-4c65-80a9-5562b0241ad4,DBO3,2018-08-13,13:02:17,3313071.0,Medium,VU,42.139089,-71.524434,Dropoff,E-1.1H,[{'package_id': 'PackageID_f42fbe13-089e-4e92-...,1.0,True,111.0
2,RouteID_f9639176-8909-4c65-80a9-5562b0241ad4,DBO3,2018-08-13,13:02:17,3313071.0,Medium,ZP,42.138518,-71.525506,Dropoff,E-1.1H,[{'package_id': 'PackageID_8fc292ae-f756-4541-...,2.0,False,90.5
3,RouteID_f9639176-8909-4c65-80a9-5562b0241ad4,DBO3,2018-08-13,13:02:17,3313071.0,Medium,KH,42.138304,-71.525361,Dropoff,E-1.1H,[{'package_id': 'PackageID_f420fbef-0cfe-4267-...,1.0,False,71.0
4,RouteID_f9639176-8909-4c65-80a9-5562b0241ad4,DBO3,2018-08-13,13:02:17,3313071.0,Medium,WW,42.138168,-71.529156,Dropoff,E-1.1H,[{'package_id': 'PackageID_e85fe820-0055-4f84-...,2.0,False,73.0


In [ ]:
fig = go.Figure(go.Scattermapbox(
    lat=route_data_ordered['lat'],
    lon=route_data_ordered['lng'],
    mode='lines',
    line=dict(width=2, color='blue'),  # Customize line color and width here
    hoverinfo='none'
))

# Add scatter points for each unique zone_id, with custom hover text
for zone_id in route_data_ordered['zone_id'].unique():
    df_sub = route_data_ordered[route_data_ordered['zone_id'] == zone_id]

    # Creating custom hover text
    hover_text = df_sub.apply(lambda row: f"Sequence: {row.name}<br> Stop ID: {row['stop_id']}<br>Zone ID: {row['zone_id']}<br>Num Packages: {row['num_packages']} <br> Service Time: {row['service_time_seconds']}", axis=1)


    fig.add_trace(go.Scattermapbox(
        lat=df_sub['lat'],
        lon=df_sub['lng'],
        mode='markers',
        marker=go.scattermapbox.Marker(size=15),
        name=str(zone_id),
        text=hover_text,
        hoverinfo='text'
    ))

# Update the layout to use OpenStreetMap style and adjust other layout properties
fig.update_layout(

    height=600,
    mapbox=dict(
        style="open-street-map",
        zoom=10,  # Adjust zoom level here
        center=dict(lat=route_data_ordered['lat'].mean(), lon=route_data_ordered['lng'].mean())
    ),
    showlegend=False  # Set to True if you want to show legend
)
fig.update_layout(margin=dict(l=10, r=10, t=10, b=10))


# Show the figure
fig.show()

In [ ]:
drone_data_ordered = pd.concat([route_data_route_df[route_data_route_df['stop_id'] == stop_id] for stop_id in test.drone_path]).reset_index(drop=True)
drone_data_ordered.head()

ValueError: No objects to concatenate

In [ ]:
fig = go.Figure(go.Scattermapbox(
    lat=drone_data_ordered['lat'],
    lon=drone_data_ordered['lng'],
    mode='lines',
    line=dict(width=2, color='blue'),  # Customize line color and width here
    hoverinfo='none'
))

# Add scatter points for each unique zone_id, with custom hover text
for zone_id in drone_data_ordered['zone_id'].unique():
    df_sub = drone_data_ordered[drone_data_ordered['zone_id'] == zone_id]

    # Creating custom hover text
    hover_text = df_sub.apply(lambda row: f"Sequence: {row.name}<br> Stop ID: {row['stop_id']}<br>Zone ID: {row['zone_id']}<br>Num Packages: {row['num_packages']} <br> Service Time: {row['service_time_seconds']}", axis=1)


    fig.add_trace(go.Scattermapbox(
        lat=df_sub['lat'],
        lon=df_sub['lng'],
        mode='markers',
        marker=go.scattermapbox.Marker(size=15),
        name=str(zone_id),
        text=hover_text,
        hoverinfo='text'
    ))

# Update the layout to use OpenStreetMap style and adjust other layout properties
fig.update_layout(

    height=600,
    mapbox=dict(
        style="open-street-map",
        zoom=10,  # Adjust zoom level here
        center=dict(lat=drone_data_ordered['lat'].mean(), lon=drone_data_ordered['lng'].mean())
    ),
    showlegend=False  # Set to True if you want to show legend
)
fig.update_layout(margin=dict(l=10, r=10, t=10, b=10))


# Show the figure
fig.show()

In [ ]:
# Create a figure
fig = go.Figure()

# Add the drone path
fig.add_trace(go.Scattermapbox(
    lat=drone_data_ordered['lat'],
    lon=drone_data_ordered['lng'],
    mode='lines',
    line=dict(width=2, color='red'),
    hoverinfo='none',
    name='Drone Path'
))

# Add scatter points for each unique zone_id in the drone data
for zone_id in drone_data_ordered['zone_id'].unique():
    df_sub = drone_data_ordered[drone_data_ordered['zone_id'] == zone_id]
    hover_text = df_sub.apply(lambda row: f"Sequence: {row.name}<br> Stop ID: {row['stop_id']}<br>Zone ID: {row['zone_id']}<br>Num Packages: {row['num_packages']} <br> Service Time: {row['service_time_seconds']}", axis=1)
    fig.add_trace(go.Scattermapbox(
        lat=df_sub['lat'],
        lon=df_sub['lng'],
        mode='markers',
        marker=go.scattermapbox.Marker(size=15),
        name=f"Drone Zone {zone_id}",
        text=hover_text,
        hoverinfo='text'
    ))


# Add the route path
fig.add_trace(go.Scattermapbox(
    lat=route_data_ordered['lat'],
    lon=route_data_ordered['lng'],
    mode='lines',
    line=dict(width=2, color='blue'),
    hoverinfo='none',
    name='Route Path'
))

# Add scatter points for each unique zone_id in the route data
for zone_id in route_data_ordered['zone_id'].unique():
    df_sub = route_data_ordered[route_data_ordered['zone_id'] == zone_id]
    hover_text = df_sub.apply(lambda row: f"Sequence: {row.name}<br> Stop ID: {row['stop_id']}<br>Zone ID: {row['zone_id']}<br>Num Packages: {row['num_packages']} <br> Service Time: {row['service_time_seconds']}", axis=1)
    fig.add_trace(go.Scattermapbox(
        lat=df_sub['lat'],
        lon=df_sub['lng'],
        mode='markers',
        marker=go.scattermapbox.Marker(size=15),
        name=f"Route Zone {zone_id}",
        text=hover_text,
        hoverinfo='text'
    ))



# Update the layout to use OpenStreetMap style and adjust other layout properties
fig.update_layout(
    height=600,
    mapbox=dict(
        style="open-street-map",
        zoom=10,
        center=dict(
            lat=(route_data_ordered['lat'].mean() + drone_data_ordered['lat'].mean()) / 2,
            lon=(route_data_ordered['lng'].mean() + drone_data_ordered['lng'].mean()) / 2
        )
    ),
    showlegend=True
)

fig.update_layout(margin=dict(l=10, r=10, t=10, b=10))

# Show the figure
fig.show()

In [50]:
proposed_sequences = json.load(open('proposed_sequences.json'))
proposed_route_ids = list(proposed_sequences.keys())

for route_id in proposed_route_ids:
    package_data_route_df, route_data_route_df, vehicle_travel_times_route = get_data_for_route(route_id)
    route_data_route_df['zone_id'].ffill(inplace=True)
    
    drone_travel_times_route = calculate_drone_travel_time(route_data_route_df)
    
    
    package_data_route_df['drone_possible'] = (package_data_route_df['depth_cm'] < 30) & (package_data_route_df['height_cm'] < 25) & (package_data_route_df['width_cm'] < 30) # TODO: be indifferent on depth and width
    
    

    package_info_agg = package_data_route_df.groupby('stop_id').apply(
    lambda x: pd.Series({
        'packages': x[['package_id', 'depth_cm', 'height_cm', 'width_cm']].to_dict('records'),
        'num_packages': len(x),
        'drone_possible': (len(x) == 1) and (x['drone_possible'].iloc[0] == True),
        'service_time_seconds': x['planned_service_time_seconds'].mean() # Not sure if this should be sum or average. https://github.com/MIT-CAVE/rc-cli/blob/main/templates/data_structures.md
    })
    ).reset_index()

    route_data_route_df = pd.merge(route_data_route_df, package_info_agg, how='left', on='stop_id')
    
    solver = VRPDSolver(route_id, package_data_route_df, route_data_route_df, vehicle_travel_times_route, drone_travel_times_route)
    solver.naive_vrpd()
    # save everything
    solver.save_vehicle_path('naive_vehicle_paths')
    solver.save_drone_path('naive_drone_paths')
    solver.save_vehicle_events('naive_vehicle_events')
    solver.save_drone_events('naive_drone_events')
    
    
